In [1]:
import os
import numpy as np
import pandas as pd
import swifter
from sklearn.model_selection import train_test_split
import utils
import catboost

In [2]:
DATA_PATH = "./data"

In [3]:
train, test = utils.load_data_csv(DATA_PATH, utils.SIMPLE_FEATURE_COLUMNS)

/root/anaconda3/envs/idao_cb/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
train.columns

Index(['ncl[0]', 'ncl[1]', 'ncl[2]', 'ncl[3]', 'avg_cs[0]', 'avg_cs[1]',
       'avg_cs[2]', 'avg_cs[3]', 'ndof', 'MatchedHit_TYPE[0]',
       'MatchedHit_TYPE[1]', 'MatchedHit_TYPE[2]', 'MatchedHit_TYPE[3]',
       'MatchedHit_X[0]', 'MatchedHit_X[1]', 'MatchedHit_X[2]',
       'MatchedHit_X[3]', 'MatchedHit_Y[0]', 'MatchedHit_Y[1]',
       'MatchedHit_Y[2]', 'MatchedHit_Y[3]', 'MatchedHit_Z[0]',
       'MatchedHit_Z[1]', 'MatchedHit_Z[2]', 'MatchedHit_Z[3]',
       'MatchedHit_DX[0]', 'MatchedHit_DX[1]', 'MatchedHit_DX[2]',
       'MatchedHit_DX[3]', 'MatchedHit_DY[0]', 'MatchedHit_DY[1]',
       'MatchedHit_DY[2]', 'MatchedHit_DY[3]', 'MatchedHit_DZ[0]',
       'MatchedHit_DZ[1]', 'MatchedHit_DZ[2]', 'MatchedHit_DZ[3]',
       'MatchedHit_T[0]', 'MatchedHit_T[1]', 'MatchedHit_T[2]',
       'MatchedHit_T[3]', 'MatchedHit_DT[0]', 'MatchedHit_DT[1]',
       'MatchedHit_DT[2]', 'MatchedHit_DT[3]', 'Lextra_X[0]', 'Lextra_X[1]',
       'Lextra_X[2]', 'Lextra_X[3]', 'Lextra_Y[0]', 'Lextra_

In [5]:
func = utils.find_closest_hit_per_station

In [6]:
# dt_train = pd.read_hdf('closest_hits_features.train.filled.p1000.hdf', 'key')
# dt_test  = pd.read_hdf('closest_hits_features.test.filled.p1000.hdf', 'key')

dt_train = pd.read_hdf('closest_hits_features.train.filled.m9999.v1.hdf', 'key')
dt_test  = pd.read_hdf('closest_hits_features.test.filled.m9999.v1.hdf', 'key')

In [7]:
dt_train.columns = [func.__name__ + '_' + str(c) if str(c).isnumeric() else c for c in dt_train.columns]
dt_test.columns  = [func.__name__ + '_' + str(c) if str(c).isnumeric() else c for c in dt_test.columns]

In [8]:
train = pd.concat([train, dt_train], axis=1, copy=False)
test = pd.concat([test, dt_test], axis=1, copy=False)

In [9]:
train.shape, test.shape

((5445705, 115), (726095, 113))

In [10]:
set(train.columns.tolist()).symmetric_difference(test.columns.tolist())

{'label', 'weight'}

In [11]:
del test

## Новые признаки

In [12]:
NA_VALUE = -9999.0

In [13]:
def fillna(df):
    """
    Пропуски только в 2 и 3 колонках
    """
    
    for axis in 'XYZ':
        cols = ['MatchedHit_{}[{}]'.format(axis, i) for i in range(4)]
        mask = np.isclose(df.loc[:, cols[2]], NA_VALUE)
        mask = mask & ~np.isclose(df.loc[:, cols[3]], NA_VALUE)
        
        df.loc[mask, cols[2]] = df.loc[mask, [cols[1], cols[3]]].mean(axis=1)
    return df

In [14]:
# train, test = map(fillna, [train, test])

In [15]:
train = fillna(train)

### Вычесть среднее из MatchedHit_Z

In [16]:
matched_hit_z_cols = train.columns.str.startswith("MatchedHit_Z")
mask = np.isclose(train.loc[:, matched_hit_z_cols], NA_VALUE).any(axis=0)
matched_hit_z_mean = train.loc[~mask, matched_hit_z_cols].mean(axis=0)
matched_hit_z_mean

MatchedHit_Z[0]    15167.8215
MatchedHit_Z[1]    16374.1045
MatchedHit_Z[2]    17567.8400
MatchedHit_Z[3]    18776.8770
dtype: float64

In [17]:
matched_hit_z_cols = train.columns[train.columns.str.startswith("MatchedHit_Z")]
matched_hit_z_mean = pd.Series([15270, 16470, 17670, 18870], index=matched_hit_z_cols)
matched_hit_z_mean

MatchedHit_Z[0]    15270
MatchedHit_Z[1]    16470
MatchedHit_Z[2]    17670
MatchedHit_Z[3]    18870
dtype: int64

In [18]:
def make_unbiased_z(df):
    for i in range(4):
        distance = np.zeros(df.shape[0])
    
        col = f"MatchedHit_Z[{i}]"
        mask = np.isclose(df.loc[:, col], NA_VALUE)
        distance[~mask] = df.loc[~mask, col] - matched_hit_z_mean[col]
        distance[mask] = NA_VALUE
    
        df[col] = distance
        
        """
        df[f"MatchedHit_Z[{i}]_sign"] = np.sign(df[col]).astype(np.int8)
        """
    return df

In [19]:
# train, test = map(make_unbiased_z, [train, test])

In [20]:
train = make_unbiased_z(train)

In [21]:
train.loc[:, train.columns.str.startswith("MatchedHit_Z")].head()

,MatchedHit_Z[0],MatchedHit_Z[1],MatchedHit_Z[2],MatchedHit_Z[3]
0,-146.955,-142.367,-149.174,-140.676
1,-57.402,-49.424,-55.146,-45.570
2,47.953,52.060,-71.700,-63.648
3,-68.590,-64.445,-71.700,-63.648
4,-64.511,-60.047,-66.979,-58.605


### Расстояние до центра (+)

In [22]:
def compute_polar_distances_matched_hit(df):
    for i in range(4):
        cols = [f"MatchedHit_X[{i}]", f"MatchedHit_Y[{i}]"]
        mask = np.isclose(df.loc[:, cols], NA_VALUE).any(axis=1)
    
        name = f"MatchedHit_R[{i}]"
        df[name] = NA_VALUE
        df.loc[~mask, name] = np.log(1.0 + np.power(df.loc[~mask, cols], 2).sum(axis=1))
        
        name = f"MatchedHit_A[{i}]"
        df[name] = NA_VALUE
        df.loc[~mask, name] = np.arctan2(df.loc[~mask, cols[1]], df.loc[~mask, cols[0]])

    # намеренное переупорядочивание признаков
    cols  = df.columns[df.columns.str.startswith("MatchedHit_R")].tolist()
    cols += df.columns[df.columns.str.startswith("MatchedHit_A")].tolist()
    cols  = df.columns[:-len(cols)].tolist() + cols
    df = df[cols]

    return df

In [23]:
# train, test = map(compute_polar_distances_matched_hit, [train, test])

In [24]:
train = compute_polar_distances_matched_hit(train)

In [25]:
train.loc[:, train.columns.str.startswith("MatchedHit_R")].head()

,MatchedHit_R[0],MatchedHit_R[1],MatchedHit_R[2],MatchedHit_R[3]
0,16.023317,16.206292,16.332808,16.583070
1,15.647796,15.811899,15.934739,16.044511
2,14.788550,14.971064,15.147750,15.280270
3,14.899596,15.022741,15.147750,15.280270
4,14.962988,15.139525,15.316049,15.449411


In [26]:
train.loc[:, train.columns.str.startswith("MatchedHit_A")].head()

,MatchedHit_A[0],MatchedHit_A[1],MatchedHit_A[2],MatchedHit_A[3]
0,2.668808,2.676832,2.672886,2.701153
1,1.270437,1.251195,1.280089,1.319231
2,-2.944427,-2.947037,-3.027307,-3.028216
3,-3.030048,-3.028084,-3.027307,-3.028216
4,2.582489,2.590782,2.601329,2.600883


In [27]:
def compute_polar_distances_lextra(df):
    for i in range(4):
        cols = [f"Lextra_X[{i}]", f"Lextra_Y[{i}]"]
        mask = np.isclose(df.loc[:, cols], NA_VALUE).any(axis=1)
    
        name = f"Lextra_R[{i}]"
        df[name] = NA_VALUE
        df.loc[~mask, name] = np.log(1.0 + np.power(df.loc[~mask, cols], 2).sum(axis=1))

        """
        name = f"Lextra_A[{i}]"
        df[name] = NA_VALUE
        df.loc[~mask, name] = np.arctan2(df.loc[~mask, cols[1]], df.loc[~mask, cols[0]])
        """
    
    # намеренное переупорядочивание признаков
    cols  = df.columns[df.columns.str.startswith("Lextra_R")].tolist()
    cols += df.columns[df.columns.str.startswith("Lextra_A")].tolist()
    cols  = df.columns[:-len(cols)].tolist() + cols
    df = df[cols]

    return df

In [28]:
# train, test = map(compute_polar_distances_lextra, [train, test])

In [29]:
train = compute_polar_distances_lextra(train)

In [30]:
train.loc[:, train.columns.str.startswith("Lextra_R")].head()

,Lextra_R[0],Lextra_R[1],Lextra_R[2],Lextra_R[3]
0,16.018055,16.192742,16.353447,16.502236
1,15.678488,15.834172,15.978674,16.113486
2,14.774766,14.959656,15.128904,15.284951
3,14.902741,15.039450,15.167410,15.287674
4,14.990755,15.168836,15.332478,15.483829


In [31]:
train.loc[:, train.columns.str.startswith("Lextra_A")].head()

""
0
1
2
3
4


### Дельты для траекторий MatchedHit_X, MatchedHit_Y

In [32]:
def diff_matched_hit(df):
    for axis in ['X', 'Y']:
        for i, j in zip([0, 1, 2, 3], [1, 2, 3, 0]):
            col_i = f"MatchedHit_{axis}[{i}]"
            col_j = f"MatchedHit_{axis}[{j}]"
            mask = np.isclose(df.loc[:, [col_i, col_j]], NA_VALUE).any(axis=1)

            name = f"MatchedHit_{axis}[{j}/{i}]_delta"
            df[name] = df[name + '_abs'] = NA_VALUE
            
            df.loc[~mask, name] = df.loc[~mask, col_j] - df.loc[~mask, col_i]
            df.loc[~mask, name + '_abs'] = np.abs(df.loc[~mask, name])
    return df

In [33]:
# train, test = map(diff_matched_hit, [train, test])

In [34]:
train = diff_matched_hit(train)

### Дельты и отношения для радиусов (+)

In [35]:
def ratio_radius_matched_hit(df):
    for i, j in zip([0, 1, 2, 3], [1, 2, 3, 0]):
        col_i = f"MatchedHit_R[{i}]"
        col_j = f"MatchedHit_R[{j}]"
        mask = np.isclose(df.loc[:, [col_i, col_j]], NA_VALUE).any(axis=1)

        name = f"MatchedHit_R[{j}/{i}]_delta"
        name_abs = name + '_abs'
        df[name] = df[name_abs] = NA_VALUE
        
        df.loc[~mask, name] = df.loc[~mask, col_j] - df.loc[~mask, col_i]
        df.loc[~mask, name_abs] = np.abs(df.loc[~mask, name])
        
        name = f"MatchedHit_R[{j}/{i}]_frac"
        df[name] = NA_VALUE
        df.loc[~mask, name] = df.loc[~mask, col_j] / df.loc[~mask, col_i]
    return df

In [36]:
# train, test = map(ratio_radius_matched_hit, [train, test])

In [37]:
train = ratio_radius_matched_hit(train)

In [38]:
cols = filter(lambda s: s.startswith("MatchedHit_R") and ("_delta" in s or "_frac" in s), train.columns)
train.loc[:, cols].head()

,MatchedHit_R[1/0]_delta,MatchedHit_R[1/0]_delta_abs,MatchedHit_R[1/0]_frac,MatchedHit_R[2/1]_delta,MatchedHit_R[2/1]_delta_abs,MatchedHit_R[2/1]_frac,MatchedHit_R[3/2]_delta,MatchedHit_R[3/2]_delta_abs,MatchedHit_R[3/2]_frac,MatchedHit_R[0/3]_delta,MatchedHit_R[0/3]_delta_abs,MatchedHit_R[0/3]_frac
0,0.182975,0.182975,1.011419,0.126516,0.126516,1.007807,0.250262,0.250262,1.015323,-0.559754,0.559754,0.966245
1,0.164103,0.164103,1.010487,0.122840,0.122840,1.007769,0.109772,0.109772,1.006889,-0.396714,0.396714,0.975274
2,0.182513,0.182513,1.012342,0.176687,0.176687,1.011802,0.132520,0.132520,1.008748,-0.491720,0.491720,0.967820
3,0.123146,0.123146,1.008265,0.125009,0.125009,1.008321,0.132520,0.132520,1.008748,-0.380674,0.380674,0.975087
4,0.176537,0.176537,1.011798,0.176523,0.176523,1.011660,0.133362,0.133362,1.008707,-0.486422,0.486422,0.968515


In [39]:
def ratio_radius_lextra_hit(df):
    for i, j in zip([0, 1, 2, 3], [1, 2, 3, 0]):
        col_i = f"Lextra_R[{i}]"
        col_j = f"Lextra_R[{j}]"
        mask = np.isclose(df.loc[:, [col_i, col_j]], NA_VALUE).any(axis=1)

        name = f"Lextra_R[{j}/{i}]_delta"
        name_abs = name + '_abs'
        df[name] = df[name_abs] = NA_VALUE
        
        df.loc[~mask, name] = df.loc[~mask, col_j] - df.loc[~mask, col_i]
        df.loc[~mask, name_abs] = np.abs(df.loc[~mask, name])
        
#         name = f"Lextra_R[{j}/{i}]_frac"
#         df[name] = NA_VALUE
#         df.loc[~mask, name] = df.loc[~mask, col_j] / df.loc[~mask, col_i]
    return df

In [40]:
# train, test = map(ratio_radius_lextra_hit, [train, test])

In [41]:
train = ratio_radius_lextra_hit(train)

In [42]:
cols = filter(lambda s: s.startswith("Lextra_R") and ("_delta" in s or "_frac" in s), train.columns)
train.loc[:, cols].head()

,Lextra_R[1/0]_delta,Lextra_R[1/0]_delta_abs,Lextra_R[2/1]_delta,Lextra_R[2/1]_delta_abs,Lextra_R[3/2]_delta,Lextra_R[3/2]_delta_abs,Lextra_R[0/3]_delta,Lextra_R[0/3]_delta_abs
0,0.174687,0.174687,0.160705,0.160705,0.148789,0.148789,-0.484181,0.484181
1,0.155684,0.155684,0.144502,0.144502,0.134812,0.134812,-0.434997,0.434997
2,0.184890,0.184890,0.169248,0.169248,0.156046,0.156046,-0.510184,0.510184
3,0.136709,0.136709,0.127960,0.127960,0.120264,0.120264,-0.384932,0.384932
4,0.178081,0.178081,0.163641,0.163641,0.151351,0.151351,-0.493073,0.493073


### Расстояние между Lextra и MatchedHit (+)

In [43]:
def distance_dist_lextra_matched(df):
    distance_counts = np.zeros(df.shape[0], dtype=int)
    
    distance_se = np.zeros(df.shape[0])
    distance_ae = np.zeros(df.shape[0])
    
    for i in range(4):
        distance = np.zeros(df.shape[0])

        cols = [f"MatchedHit_X[{i}]", f"MatchedHit_Y[{i}]", f"Lextra_X[{i}]", f"Lextra_Y[{i}]"]
        mask = np.isclose(df.loc[:, cols], NA_VALUE).any(axis=1)

        mse = df.loc[~mask, cols[:2]].values - df.loc[~mask, cols[2:]].values
        mse = np.power(mse, 2.0).sum(axis=1)
        
        mae = df.loc[~mask, cols[:2]].values - df.loc[~mask, cols[2:]].values
        mae = np.abs(mae).max(axis=1)

        distance[~mask] = np.log(1.0 + mse)
        distance[mask] = NA_VALUE
        
        distance_se[~mask] += mse
        distance_ae[~mask] = np.maximum(distance_ae[~mask], mae)
        distance_counts[~mask] += 1

        df[f"distance_LextraMatchedHit[{i}]"] = distance

    distance_se /= distance_counts
    df[f"distance_LextraMatchedHit_SE"] = distance_se
    df[f"distance_LextraMatchedHit_AE"] = distance_ae

    return df

In [44]:
# train, test = map(distance_dist_lextra_matched, [train, test])

In [45]:
train = distance_dist_lextra_matched(train)

In [46]:
train.loc[:, train.columns.str.startswith("distance_LextraMatchedHit")].head()

,distance_LextraMatchedHit[0],distance_LextraMatchedHit[1],distance_LextraMatchedHit[2],distance_LextraMatchedHit[3],distance_LextraMatchedHit_SE,distance_LextraMatchedHit_AE
0,6.712163,7.512178,7.232589,10.468368,9804.277406,186.32360
1,7.448349,7.150894,9.267218,11.013328,18562.759955,243.12120
2,7.192176,7.376499,9.402387,9.477220,7024.772423,113.72097
3,5.449827,6.342438,6.628810,5.935092,482.888433,21.25670
4,8.099670,8.254467,8.448039,8.426182,4091.272154,67.40200


### Абсолютное и относительное смещение

In [47]:
def offset_matched_hit(df):
    offset_abs = np.sqrt(np.power(df.loc[:, ["MatchedHit_X[0]", f"MatchedHit_Y[0]"]], 2).sum(axis=1))
    
    for i, j in zip([0, 1, 2], [1, 2, 3]):
        cols = [f"MatchedHit_X[{i}]", f"MatchedHit_Y[{i}]", f"MatchedHit_X[{j}]", f"MatchedHit_Y[{j}]"]
        mask = np.isclose(df.loc[:, cols], NA_VALUE).any(axis=1)
        
        mse = df.loc[~mask, cols[:2]].values - df.loc[~mask, cols[2:]].values
        mse = np.sqrt(np.power(mse, 2.0).sum(axis=1))
        offset_abs[~mask] += mse
        
    offset_from = df.loc[:, ["MatchedHit_X[0]", f"MatchedHit_Y[0]"]].values
    offset_to = np.zeros(shape=(df.shape[0], 2))
    
    for i in range(4):
        cols = [f"MatchedHit_X[{i}]", f"MatchedHit_Y[{i}]"]
        mask = np.isclose(df.loc[:, cols], NA_VALUE).any(axis=1)
        offset_to[~mask] = df.loc[~mask, cols].values
    
    offset_rel = np.sqrt(np.power(offset_from - offset_to, 2.0).sum(axis=1))
    
    df["MatchedHit_offset_abs"] = offset_abs
    df["MatchedHit_offset_rel"] = offset_rel
    df["MatchedHit_offset_rel_div_abs"] = offset_rel / offset_abs
    
    return df

In [48]:
# train, test = map(offset_matched_hit, [train, test])

In [49]:
train = offset_matched_hit(train)

In [50]:
train.loc[:, train.columns.str.startswith("MatchedHit_offset_")].head()

,MatchedHit_offset_abs,MatchedHit_offset_rel,MatchedHit_offset_rel_div_abs
0,4003.291053,980.480698,0.244919
1,3109.043351,564.706574,0.181634
2,2137.837527,478.845064,0.223986
3,2080.101597,360.525854,0.173321
4,2265.580966,490.083268,0.216317


### Отношение дисперсий

In [51]:
def ratio_mextra_matched(df):    
    for i in range(4):
        cols = [f"MatchedHit_DX[{i}]", f"MatchedHit_DY[{i}]", f"Mextra_DX2[{i}]", f"Mextra_DY2[{i}]"]
        mask = np.isclose(df.loc[:, cols], NA_VALUE).any(axis=1)

        tmpX = df.loc[~mask, cols[::2]]
        tmpY = df.loc[~mask, cols[1::2]]
        
        ratio = tmpX.iloc[:, 0] / np.sqrt(tmpX.iloc[:, 1]) + \
                tmpY.iloc[:, 0] / np.sqrt(tmpY.iloc[:, 1])

        name = f"ratio_MextraMatchedHit[{i}]"
        df[name] = NA_VALUE
        df.loc[~mask, name] = ratio

    return df

In [52]:
# train, test = map(ratio_mextra_matched, [train, test])

In [53]:
train = ratio_mextra_matched(train)

In [54]:
train.loc[:, train.columns.str.startswith("ratio_MextraMatchedHit")].head()

,ratio_MextraMatchedHit[0],ratio_MextraMatchedHit[1],ratio_MextraMatchedHit[2],ratio_MextraMatchedHit[3]
0,8.287385,5.293275,5.687804,4.349901
1,18.965682,12.044753,13.049134,32.661490
2,6.001871,3.893571,4.204257,3.224819
3,12.649516,8.039806,8.694506,6.719570
4,5.487896,3.481615,3.740265,2.870688


### Левые и правые границы для Lextra (- не влезает в память)

In [55]:
def sigma_lextra_hit(df):
    for axis in ['X', 'Y']:
        for i in range(0, 4):
            col_i = f"Lextra_{axis}[{i}]"
            col_j = f"Mextra_D{axis}2[{i}]"
            mask = np.isclose(df.loc[:, [col_i, col_j]], NA_VALUE).any(axis=1)

            for col, factor in zip([f"Lextra_{axis}[{i}]_minus_sigma2",
                                    f"Lextra_{axis}[{i}]_minus_sigma",
                                    f"Lextra_{axis}[{i}]_plus_sigma",
                                    f"Lextra_{axis}[{i}]_plus_sigma2"],
                                   [-2, -1, 1, 2]):
                df[col] = NA_VALUE
                df.loc[~mask, col] = df.loc[~mask, col_i] + factor * np.sqrt(df.loc[~mask, col_j])
        
    return df

### Число объектов в кластерах

In [56]:
def cluster_size(df):
    for i in range(4):
        name = f'cl_size[{i}]'
        df[name] = NA_VALUE
        
        cols = [f'ncl[{i}]', f'avg_cs[{i}]']
        mask = np.isclose(df.loc[:, cols], NA_VALUE).any(axis=1)
        df.loc[~mask, name] = df.loc[~mask, cols[0]] * df.loc[~mask, cols[1]]
    return df

In [57]:
# train, test = map(cluster_size, [train, test])

In [58]:
train = cluster_size(train)

In [59]:
train.loc[:, train.columns.str.startswith('cl_size')].head()

,cl_size[0],cl_size[1],cl_size[2],cl_size[3]
0,94.000000,49.000001,14.999999,17.000000
1,253.000000,53.000002,15.000000,32.000002
2,293.000000,51.000001,12.999999,13.000001
3,227.000002,67.999997,25.000000,26.000000
4,33.000001,43.000001,9.000000,14.000000


### Последняя площадка

In [60]:
def station_latest(df):
    station = np.zeros(df.shape[0], dtype=int)
    
    for i in range(4):
        cols = [f"MatchedHit_X[{i}]", f"MatchedHit_Y[{i}]", f"MatchedHit_Z[{i}]"]
        mask = np.isclose(df.loc[:, cols], NA_VALUE).any(axis=1)
        station[~mask] += 1
        
    df["station_latest"] = station
    
    return df

In [61]:
# train, test = map(station_latest, [train, test])

In [62]:
# train.loc[:, "station_latest"].head()

In [63]:
# np.unique(train['station_latest'], return_counts=True)

### Еще один момент

In [64]:
def momentum(df):
    mask = np.isclose(df.loc[:, ['P', 'PT']], NA_VALUE).any(axis=1)
    df['PO'] = NA_VALUE
    df.loc[~mask, 'PO'] = np.power(df.loc[~mask, 'P'], 2.0) - np.power(df.loc[~mask, 'PT'], 2.0)
    return df

In [65]:
# train, test = map(momentum, [train, test])

In [66]:
train = momentum(train)

In [67]:
train.loc[:, ['P', 'PT', 'PO']].head()

,P,PT,PO
0,12646.817591,1764.925899,1.568270e+08
1,28556.630027,4511.281266,7.951295e+08
2,17491.702634,1062.550086,3.048306e+08
3,37192.868559,4991.074825,1.358399e+09
4,16688.746174,1347.704888,2.766979e+08


### Отношение NShared к FOI_hits_N

In [68]:
def nshared_foihits(df):
    df.loc[:, "frac_NShared_FOI_hits_N"] = df.loc[:, "NShared"] / df.loc[:, "FOI_hits_N"] 
    df.loc[:, "frac_NShared_FOI_hits_N"] = df.loc[:, "frac_NShared_FOI_hits_N"].fillna(-1)
    return df

In [69]:
# train, test = map(nshared_foihits, [train, test])

In [70]:
train = nshared_foihits(train)

In [71]:
train.loc[:, ["NShared", "FOI_hits_N", "frac_NShared_FOI_hits_N"]].head()

,NShared,FOI_hits_N,frac_NShared_FOI_hits_N
0,0,6,0.00
1,1,4,0.25
2,0,9,0.00
3,0,5,0.00
4,0,6,0.00


### Усредняем дисперсии

In [72]:
def mean_std(df):
    for station in range(4):
        cols = [f'MatchedHit_D{axis}[{station}]' for axis in 'XYZ']
        mask = np.isclose(df.loc[:, cols], -1).any(axis=1)
        
        name = f'MatchedHit_D[{station}]_mean'
        df[name] = -1
        df.loc[~mask, name] = df.loc[~mask, cols].mean(axis=1)
        
    for station in range(4):
        cols = [f'Mextra_D{axis}2[{station}]' for axis in 'XY']
        mask = np.isclose(df.loc[:, cols], -1).any(axis=1)
        
        name = f'Mextra_D2[{station}]_mean'
        df[name] = -1
        df.loc[~mask, name] = df.loc[~mask, cols].mean(axis=1)

    return df

In [73]:
# train, test = map(mean_std, [train, test])

In [74]:
train = mean_std(train)

### Удаляем лишние колонки

In [75]:
import operator
from functools import reduce

def drop_columns(df):
    mask = [df.columns.str.startswith('MatchedHit_X'),
            df.columns.str.startswith('MatchedHit_Y'),
            df.columns.str.startswith('Lextra_X'),
            df.columns.str.startswith('Lextra_Y'),
            df.columns.str.startswith('FOI_hits_N['),
            df.columns == "ndof",
            df.columns.str.startswith('MatchedHit_DX') & ~df.columns.str.endswith('_mean'),
            df.columns.str.startswith('MatchedHit_DY') & ~df.columns.str.endswith('_mean'),
            df.columns.str.startswith('MatchedHit_DZ') & ~df.columns.str.endswith('_mean'),
            df.columns.str.startswith('Mextra_D') & ~df.columns.str.endswith('_mean')]
    mask = reduce(operator.or_, mask)
    df.drop(columns=df.columns[mask], inplace=True)
    return df

In [76]:
# train, test = map(drop_columns, [train, test])

In [77]:
train = drop_columns(train)

## Обучение модели

I know this is incorrect. See it as a low-hanging fruit to beat the baseline. Catboost affirmatively [refused](https://github.com/catboost/catboost/pull/399) to support negative weights. At the same time, its evaluation is [extremly fast](https://catboost.ai/news/best-in-class-inference-and-a-ton-of-speedups), so we are using it as the timing benchmark for Track 2. Feel free to use a patched version with disabled negative weights check.

In [78]:
features = list(filter(lambda s: s not in utils.TRAIN_COLUMNS, train.columns))
features

['ncl[0]',
 'ncl[1]',
 'ncl[2]',
 'ncl[3]',
 'avg_cs[0]',
 'avg_cs[1]',
 'avg_cs[2]',
 'avg_cs[3]',
 'MatchedHit_TYPE[0]',
 'MatchedHit_TYPE[1]',
 'MatchedHit_TYPE[2]',
 'MatchedHit_TYPE[3]',
 'MatchedHit_Z[0]',
 'MatchedHit_Z[1]',
 'MatchedHit_Z[2]',
 'MatchedHit_Z[3]',
 'MatchedHit_T[0]',
 'MatchedHit_T[1]',
 'MatchedHit_T[2]',
 'MatchedHit_T[3]',
 'MatchedHit_DT[0]',
 'MatchedHit_DT[1]',
 'MatchedHit_DT[2]',
 'MatchedHit_DT[3]',
 'NShared',
 'FOI_hits_N',
 'PT',
 'P',
 'find_closest_hit_per_station_0',
 'find_closest_hit_per_station_1',
 'find_closest_hit_per_station_2',
 'find_closest_hit_per_station_3',
 'find_closest_hit_per_station_4',
 'find_closest_hit_per_station_5',
 'find_closest_hit_per_station_6',
 'find_closest_hit_per_station_7',
 'find_closest_hit_per_station_8',
 'find_closest_hit_per_station_9',
 'find_closest_hit_per_station_10',
 'find_closest_hit_per_station_11',
 'find_closest_hit_per_station_12',
 'find_closest_hit_per_station_13',
 'find_closest_hit_per_station

In [79]:
%%time

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score, log_loss
from scoring import rejection90

models_g = []
scores_g = []

# kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=4986)
kf = KFold(n_splits=5, shuffle=True, random_state=4986)

np.random.seed(5872)
seeds = np.random.randint(1_000, 10_000, size=5)
print("seeds:", *seeds)

for train_i, valid_i in kf.split(train):
    X_train, w_train, y_train = train.loc[train_i, features], \
        train.loc[train_i, 'weight'], train.loc[train_i, 'label']
    X_valid, w_valid, y_valid = train.loc[valid_i, features], \
        train.loc[valid_i, 'weight'], train.loc[valid_i, 'label']
    
    X_train, w_train, y_train = map(lambda o: o.values, [X_train, w_train, y_train])
    X_valid, w_valid, y_valid = map(lambda o: o.values, [X_valid, w_valid, y_valid])
    
    model = catboost.CatBoostClassifier(
        iterations=7000, max_depth=8, thread_count=16,
        verbose=False, task_type='GPU', random_state=seeds[len(models_g)]
    )

    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)],
              sample_weight=np.abs(w_train))
    
    y_pred = model.predict_proba(X_valid)[:,1]
    
    score = {
        'log_loss': log_loss(y_valid, y_pred),
        'auc': roc_auc_score(y_valid, y_pred),
        'rejection90': rejection90(y_valid, y_pred, sample_weight=w_valid)
    }
    
    print("log_loss = {log_loss:.6f}; auc = {auc:.6f}; rejection90 = {rejection90:.6f}".format(**score))
    
    models_g.append(model)
    scores_g.append(score)

seeds: 1925 7354 7914 4109 2190
log_loss = 0.401340; auc = 0.850647; rejection90 = 0.800087
log_loss = 0.402666; auc = 0.852841; rejection90 = 0.792045
log_loss = 0.401943; auc = 0.851426; rejection90 = 0.777426
log_loss = 0.401953; auc = 0.852565; rejection90 = 0.776362
log_loss = 0.402289; auc = 0.852336; rejection90 = 0.780848
CPU times: user 2h 49min 35s, sys: 1h 15min 27s, total: 4h 5min 2s
Wall time: 1h 24min 14s


In [80]:
from collections import defaultdict
from functools import reduce
from operator import setitem

score = reduce(lambda d, sc: any(setitem(d, k, d[k] + v) for k, v in sc.items()) or d,
               scores_g, defaultdict(float))

for k, v in score.items():
    score[k] = v / len(scores_g)
    
print("log_loss = {log_loss:.6f}; auc = {auc:.6f}; rejection90 = {rejection90:.6f}".format(**score))

log_loss = 0.402038; auc = 0.851963; rejection90 = 0.785354


In [81]:
score['rejection90']

0.7853535945174174

In [82]:
feature_importances = np.mean([np.asarray(model.feature_importances_) / sum(model.feature_importances_)
                               for model in models_g], axis=0)
sorted(list(zip(features, feature_importances)), key=lambda f: f[-1], reverse=True)

[('PT', 0.055846537595657685),
 ('distance_LextraMatchedHit_SE', 0.03164172409569899),
 ('distance_LextraMatchedHit_AE', 0.03157313509792627),
 ('find_closest_hit_per_station_1', 0.023532799279253773),
 ('find_closest_hit_per_station_0', 0.023441631638413257),
 ('NShared', 0.01933434438875744),
 ('frac_NShared_FOI_hits_N', 0.019062768634130464),
 ('ncl[0]', 0.015625926391090027),
 ('P', 0.015411070397360121),
 ('cl_size[0]', 0.015366471275706797),
 ('MatchedHit_R[3]', 0.015167021153626243),
 ('ncl[1]', 0.01462885029264158),
 ('find_closest_hit_per_station_28', 0.01462500044481362),
 ('avg_cs[1]', 0.014453300899044943),
 ('find_closest_hit_per_station_29', 0.014259621279393193),
 ('ncl[2]', 0.013531251246393605),
 ('PO', 0.013352403729647539),
 ('avg_cs[2]', 0.013216236175999336),
 ('MatchedHit_R[2/1]_delta_abs', 0.013146490417476796),
 ('avg_cs[3]', 0.01313512670931237),
 ('ncl[3]', 0.012861924289614598),
 ('MatchedHit_R[1/0]_delta_abs', 0.012820317126689726),
 ('avg_cs[0]', 0.01270901

## Сохраняем модельки

In [83]:
!mkdir -p models

In [84]:
for i, model in enumerate(models_g):
    model.save_model(f"models/catboost_model_{i}.cbm")